## Summary

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

## Import Datasets

In [2]:
enron_df = pd.read_csv('datasets/explored/enron-emails-explored.csv')
mal_df = pd.read_csv('datasets/explored/malicious-emails-explored.csv')